In [1]:
import numpy as np #numpy for random number generation used for generating randomized puzzles

In [1]:
def has_dupes(list):
    for i in range(1, 10):
        if list.count(i) <= 1:
            continue
        else:
            return True
    return False

def flip_board(board):
    l = []
    for i in range(len(board)):
        l.append([board[t][i] for t in range(len(board))])
    return l

def block_board(board):
    inc = [0,3,6]
    new_board = [[] for _ in range(9)]
    index_1 = 0
    index_2 = 0
    for p in range(9):
        if p%3 == 0 and not p == 0:
            index_2 = 0
            index_1 += 3
        for t in range(3):
            for i in range(3):
                new_board[p].append(board[index_1+t][index_2+i])
        index_2 += 3
    return new_board

def is_valid(board):
    l = len(board)
    for t in range(l):
        if has_dupes(board[t]):
            return False
        elif has_dupes(flip_board(board)[t]):
            return False
        elif has_dupes(block_board(board)[t]):
            return False
    return True

def is_complete(board):
    for i in board:
        if 0 in i:
            return False
    return True

def is_solved(board):
    return is_complete(board) and is_valid(board)

def swap_rows(board, x, y):
    temp = [i for i in board[x]]
    board[x] = board[y]
    board[y] = temp
    
def swap_triple_rows(board, x, y):
    for i in range(3):
        swap_rows(board, x * 3 + i, y * 3 + i)
        
def swap_columns(board, x, y):
    temp = [board[i][x] for i in range(9)]
    for i in range(9):
        board[i][x] = board[i][y]
        board[i][y] = temp[i]

def swap_triple_columns(board, x, y):
    for i in range(3):
        swap_columns(board, x * 3 + i, y * 3 + i)

def randomized_board():
    board = [[4, 3, 1, 6, 7, 9, 5, 2, 8],
             [9, 6, 7, 2, 5, 8, 3, 4, 1],
             [5, 8, 2, 1, 4, 3, 9, 6, 7],
             [6, 5, 9, 8, 1, 7, 2, 3, 4],
             [3, 2, 8, 5, 6, 4, 1, 7, 9],
             [7, 1, 4, 9, 3, 2, 8, 5, 6],
             [8, 7, 3, 4, 2, 1, 6, 9, 5],
             [1, 4, 5, 3, 9, 6, 7, 8, 2],
             [2, 9, 6, 7, 8, 5, 4, 1, 3]]
    for i in range(10):
        method = np.random.randint(1,5)
        if method == 1:
            x = np.random.randint(9)
            swap_rows(board, x, np.random.choice([int(x/3)*3 + (x-2)%3, int(x/3)*3 + (x-1)%3]))
        elif method == 2:
            x = np.random.randint(9)
            swap_columns(board, x, np.random.choice([int(x/3)*3 + (x-2)%3, int(x/3)*3 + (x-1)%3]))
        elif method == 3:
            x = np.random.randint(3)
            swap_triple_rows(board, x, np.random.choice([int(x/3)*3 + (x-2)%3, int(x/3)*3 + (x-1)%3]))
        elif method == 4:
            x = np.random.randint(3)
            swap_triple_columns(board, x, np.random.choice([x//3*3 + (x-2)%3, x//3*3 + (x-1)%3]))
        return board
            
def unsolved_board(missing_pieces):
    coords = []
    board = randomized_board()
    for i in range(missing_pieces):
        coords.append([np.random.randint(9), np.random.randint(9)])
    for t in coords:
        board[t[0]][t[1]] = 0
    return board

def empty_spaces(board):
    coords = []
    for i in range(9):
        for t in range(9):
            if board[i][t] == 0:
                coords.append([i, t])
    return coords
    
def not_in_row(l, k):
    uncommon = []
    for i in range(len(l)):
        if l[i] not in k:
            uncommon.append(l[i])
    return uncommon

def possibilities(board, empty):
    n_list = []
    flipped = flip_board(board)
    blocked = block_board(board)
    for i in empty:
        number_list = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        number_list = not_in_row(number_list, board[i[0]])
        number_list = not_in_row(number_list, flipped[i[1]])
        index = block_coords(i)
        number_list = not_in_row(number_list, blocked[index])
        n_list.append(number_list)
    eliminate_row(n_list, empty)
    eliminate_col(n_list, empty)
    return n_list

def remove_from_list(l, num):
    if num in l:
        l.pop(index_of(l, num))
    
def row_possibilities(possible, coords, row):
    l = []
    for i in range(len(possible)):
        if coords[i][0] == row:
            for t in possible[i]:
                l.append(t)
    return l

def col_possibilities(possible, coords, col):
    l = []
    for i in range(len(possible)):
        if coords[i][1] == col:
            for t in possible[i]:
                l.append(t)
    return l

def eliminate_from_block(possible, coords, block, exception, num, row):
    for i in range(len(possible)):
        if block_coords(coords[i]) == block and not coords[i][0] == exception and row:
            remove_from_list(possible[i], num)
        elif block_coords(coords[i]) == block and not coords[i][1] == exception and not row:
            remove_from_list(possible[i], num)
        
def eliminate_row(possible, coords):
    arr = []
    for i in range(9):
        arr.append([])
        possibilities = row_possibilities(possible, coords, i)
        for t in range(1, 10):
            arr[-1].append(possibilities.count(t))
    new_arr = []
    
    for i in range(9):
        new_arr.append([])
        for t in range(1, 10):
            if arr[i][t-1] < 4 and arr[i][t-1] > 0:
                new_arr[-1].append(t)
    new_new_arr = []
    for i in range(9):
        new_new_arr.append([])
        for t in new_arr[i]:
            counter = 0
            for _ in possible:
                if coords[counter][0] == i and t in _:
                    break
                counter += 1
            block = block_coords(coords[counter])
            eliminate = True
            for e in range(len(possible)):
                if coords[e][0] == i and t in possible[e]:
                    if not block == block_coords(coords[e]):
                        eliminate = False
                    block = block_coords(coords[e])
            if eliminate:
                new_new_arr[-1].append([t, block])
    
    for i in range(len(new_new_arr)):
        for t in new_new_arr[i]:
            eliminate_from_block(possible, coords, t[1], i, t[0], True)

def eliminate_col(possible, coords):
    arr = []
    for i in range(9):
        arr.append([])
        possibilities = col_possibilities(possible, coords, i)
        for t in range(1, 10):
            arr[-1].append(possibilities.count(t))
    new_arr = []
    
    for i in range(9):
        new_arr.append([])
        for t in range(1, 10):
            if arr[i][t-1] < 4 and arr[i][t-1] > 0:
                new_arr[-1].append(t)
    new_new_arr = []
    for i in range(9):
        new_new_arr.append([])
        for t in new_arr[i]:
            counter = 0
            for _ in possible:
                if coords[counter][1] == i and t in _:
                    break
                counter += 1
            block = block_coords(coords[counter])
            eliminate = True
            for e in range(len(possible)):
                if coords[e][1] == i and t in possible[e]:
                    if not block == block_coords(coords[e]):
                        eliminate = False
                    block = block_coords(coords[e])
            if eliminate:
                new_new_arr[-1].append([t, block])
                
    for i in range(len(new_new_arr)):
        for t in new_new_arr[i]:
            eliminate_from_block(possible, coords, t[1], i, t[0], False)
            
def index_of(arr, x):
    counter = 0
    for i in arr:
        if i == x:
            break
        counter += 1
    return counter

def hidden_rows(possible, coords):
    l = [i for i in possible]
    for i in range(len(possible)):
        for t in range(len(possible)):
            if coords[t][0] == coords[i][0] and not t == i:
                l[i] = not_in_row(l[i], possible[t])
    return l

def hidden_cols(possible, coords):
    l = [i for i in possible]
    for i in range(len(possible)):
        for t in range(len(possible)):
            if coords[t][1] == coords[i][1] and not t == i:
                l[i] = not_in_row(l[i], possible[t])
    return l

def block_coords(coords):
    index = coords[0] - coords[0] % 3 + coords[1] // 3
    return index

def hidden_blocks(possible, coords):
    l = [i for i in possible]
    for i in range(len(possible)):
        for t in range(len(possible)):
            if block_coords(coords[t]) == block_coords(coords[i]) and not t == i:
                l[i] = not_in_row(l[i], possible[t])
    return l

def fill_hidden(board, possible, empty):
    hidden = [hidden_rows(possible, empty), hidden_cols(possible, empty), hidden_blocks(possible, empty)]
    # [rows, columns, blocks]
    for i in hidden:
        for t in range(len(empty)):
            if len(i[t]) == 1:
                board[empty[t][0]][empty[t][1]] = i[t][0]

def fill(board):
    coords = empty_spaces(board)
    possible_vals = possibilities(board, coords)
    lengths = [len(i) for i in possible_vals]
    for i in range(len(coords)):
        if len(possible_vals[i]) == 1:
            board[coords[i][0]][coords[i][1]] = possible_vals[i][0]
    fill_hidden(board, possible_vals, coords)
    
def print_notes(coords, possible_vals):
    arr = [[0,0,0,0,0,0,0,0,0] for i in range(9)]
    for i in range(len(coords)):
        arr[coords[i][0]][coords[i][1]] = possible_vals[i]
    for i in arr:
        print('')
        for t in i:
            if type(t) == list:
                for _ in t:
                    print(_, end='')
                print(".", end='')
                print(" " * (9 - len(t)), end='')
            else:
                print('-', " " * 8, end= '')

def solve_board(board):
    temp1 = [[t for t in i] for i in board]
    temp = [[t for t in i] for i in temp1]
    while not is_solved(temp1):
        temp = [[t for t in i] for i in temp1]
        fill(temp1)
        if not is_valid(temp1):
            break
        if temp == temp1:
            temp1 = backtrack(temp1)
            break
    if is_solved(temp1):
        return temp1
    else:
        return temp

def backtrack(board):
    coords = empty_spaces(board)
    possible_vals = possibilities(board, coords)
    lengths = [len(i) for i in possible_vals]
    if 0 in lengths:
        return board
    min_index = index_of(lengths, min(lengths))
    board_copy = [[t for t in i] for i in board]
    
    for i in possible_vals[min_index]:
        board_copy[coords[min_index][0]][coords[min_index][1]] = i
        board_copy = solve_board(board_copy)
        if is_solved(board_copy):
            return board_copy
        else:
            board_copy = [[t for t in i] for i in board]
    return board
    

In [2]:
board = [[0, 0, 0, 0, 6, 0, 5, 0, 0],
         [0, 0, 9, 0, 0, 0, 7, 0, 0],
         [1, 0, 0, 0, 0, 2, 0, 6, 9],
         [0, 0, 0, 0, 0, 0, 9, 0, 0],
         [0, 0, 1, 0, 7, 0, 0, 8, 4],
         [2, 0, 0, 3, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 5, 0],
         [0, 8, 0, 0, 0, 7, 0, 0, 0],
         [0, 0, 4, 0, 8, 0, 0, 1, 6]]  #evil puzzle copied from sudoku.com

#board = unsolved_board(50) # randomly generates a board
board #prints the board

[[0, 0, 0, 0, 6, 0, 5, 0, 0],
 [0, 0, 9, 0, 0, 0, 7, 0, 0],
 [1, 0, 0, 0, 0, 2, 0, 6, 9],
 [0, 0, 0, 0, 0, 0, 9, 0, 0],
 [0, 0, 1, 0, 7, 0, 0, 8, 4],
 [2, 0, 0, 3, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 5, 0],
 [0, 8, 0, 0, 0, 7, 0, 0, 0],
 [0, 0, 4, 0, 8, 0, 0, 1, 6]]

In [3]:
empty_cells = empty_spaces(board) #empty spaces
possible_values = possibilities(board, empty_cells) #possible values for each empty space
print_notes(empty_cells, possible_values) #print the possible values in the same format as its board


3478.     2347.     2378.     14789.    -         13489.    -         234.      1238.     
34568.    23456.    -         1458.     1345.     13458.    -         234.      1238.     
-         3457.     3578.     4578.     345.      -         348.      -         -         
345678.   34567.    35678.    124568.   1245.     14568.    -         237.      235.      
3569.     3569.     -         2569.     -         569.      236.      -         -         
-         45679.    5678.     -         1459.     145689.   16.       7.        5.        
369.      12369.    236.      2469.     2349.     3469.     2348.     -         2378.     
3569.     -         2356.     124569.   123459.   -         234.      2349.     23.       
3579.     23579.    -         259.      -         359.      23.       -         -         

In [4]:
board = solve_board(board) #the function used to solve boards
print(is_solved(board)) #prints if the given board is solved or not
board #prints the solved board

True


[[8, 2, 3, 7, 6, 9, 5, 4, 1],
 [6, 5, 9, 4, 3, 1, 7, 2, 8],
 [1, 4, 7, 8, 5, 2, 3, 6, 9],
 [4, 7, 5, 6, 1, 8, 9, 3, 2],
 [9, 3, 1, 2, 7, 5, 6, 8, 4],
 [2, 6, 8, 3, 9, 4, 1, 7, 5],
 [3, 1, 2, 9, 4, 6, 8, 5, 7],
 [5, 8, 6, 1, 2, 7, 4, 9, 3],
 [7, 9, 4, 5, 8, 3, 2, 1, 6]]